# Task 1 ( Fine-Tuning CNN Dailymail Dataset )

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/THExt-ensemble-main

In [ ]:
!pip install -r requirements.txt

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
from finetuning import finetuning
import pandas as pd
import rouge
import numpy as np
from datasets import load_dataset
from Thext import SentenceRankerPlus
from Thext import Highlighter
from Thext import RedundancyManager
import nltk
nltk.download('punkt')
nltk.download('stopwords')


In [ ]:
finetuning("dataset_fine_tuning_THExt.csv", "checkpoint")

**Evaluate**

In [ ]:
def evaluate(text, hs,  sentences):

    predicted_highlights_concat = ' '.join(map(str, sentences))
    real_highlights_concat =  hs

    r_computer = rouge.Rouge(metrics=['rouge-n', 'rouge-l'], limit_length=False, max_n=2, alpha=0.5, stemming=False)
    score = r_computer.get_scores(predicted_highlights_concat,real_highlights_concat) 

    return score['rouge-1']['f'],score['rouge-2']['f'], score['rouge-l']['f']

In [ ]:
data = load_dataset("cnn_dailymail" ,"3.0.0", split="validation")
data = pd.DataFrame(data).iloc[:1000]

In [ ]:
model_name_or_path='checkpoint' 
base_model_name = "morenolq/thext-cs-scibert"

In [ ]:
sr = SentenceRankerPlus(device='cuda')
sr.load_model(base_model_name=base_model_name, model_name_or_path=model_name_or_path,device='cuda')
rm = RedundancyManager()
h = Highlighter(sr, redundancy_manager = rm)

In [ ]:
r1_f = np.array([])
r2_f = np.array([])
rl_f = np.array([])

In [ ]:
for i in range(len(data)):
  text = data.iloc[i]['article']
  highlights = data.iloc[i]['highlights']

  sum = h.get_highlights_simple(text, abstract = True, rel_w=1.0, pos_w=0.0, red_w=0.0, prefilter=False, NH = 3)


  r1f,r2f,rlf = evaluate(text, highlights, sentences = sum)

  r1_f = np.append(r1_f,r1f)
  r2_f = np.append(r2_f,r2f)
  rl_f = np.append(rl_f,rlf)

In [ ]:
print(f"Avarage Rougue-1 f1 score : { np.average(r1_f) }")
print(f"Avarage Rougue-2 f1 score : { np.average(r2_f) }")
print(f"Avarage Rougue-l f1 score : { np.average(rl_f) }")

# Task 2 ( Ensemble method )

In [6]:
from datasets import load_dataset
import pandas as pd
from utils import Ensemble
from datasets import load_dataset
import rouge
import pandas as pd
import numpy as np

**Fit models**

In [ ]:
from sklearn.linear_model import LassoCV

random_ensamble = Ensemble("RandomForest")
sgd_ensamble = Ensemble("sgd")
lasso_ensamble = Ensemble(model = LassoCV(cv=5, random_state=0))

data = pd.read_csv("data_train.csv") 

X = data[['text_rank', 'lsa_score', 'tf_idf', 'relevance_score', 'thext_score', 'pos_i']]
y = data['rouge_2f']

random_ensamble.train(X,y)
sgd_ensamble.train(X,y)
lasso_ensamble.train(X,y)

random_ensamble.save("random")
sgd_ensamble.save("sgd")
lasso_ensamble.save("lasso")



**Test models**

In [ ]:
#random_ensamble.load("random")
#sgd_ensamble.load("sgd")
#lasso_ensamble.load("lasso")

data = load_dataset("cnn_dailymail" ,"3.0.0", split="validation")
data = pd.DataFrame(data).iloc[:1000]

In [ ]:
def evaluate(model, data):
	r1_f = []
	r2_f = []
	rl_f = []
	for i in range(len(data)):
		text = data.iloc[i]['article']
		highlights = data.iloc[i]['highlights']
		r1f,r2f,rlf = model.evaluate(text, highlights)
		r1_f.append(r1f)
		r2_f.append(r2f)
		rl_f.append(rlf)
	print(f"Avarage Rougue-1 f1 score : { np.average(r1_f) }")
	print(f"Avarage Rougue-2 f1 score : { np.average(r2_f) }")
	print(f"Avarage Rougue-l f1 score : { np.average(rl_f) }")

In [ ]:
evaluate(random_ensamble, data)

In [ ]:
evaluate(sgd_ensamble, data)

In [ ]:
evaluate(lasso_ensamble, data)

**Independent set method**

In [ ]:
from utils import RedundancyIndipendentSet

In [ ]:
random_ensamble.load("random")

data = load_dataset("cnn_dailymail" ,"3.0.0", split="validation")
data = pd.DataFrame(data).iloc[:1000]

In [ ]:
r = RedundancyIndipendentSet()

r1_f = np.array([])
r2_f = np.array([])
rl_f = np.array([])
r1_f_ind = np.array([])
r2_f_ind = np.array([])
rl_f_ind = np.array([])

In [ ]:
for i in range(len(data)):

  text = data.iloc[i]['article']
  highlights = data.iloc[i]['highlights']

  sum = random_ensamble.summary(text, NH = 3)
  sum_redundancy = random_ensamble.summary(text, NH = 5, score=True)
  ind = r.indipendent_set(sum_redundancy)

  r1f_ind,r2f_ind,rlf_ind = random_ensamble.evaluate(text, highlights, sent = ind )
  r1f,r2f,rlf = random_ensamble.evaluate(text, highlights, sent = sum)

  r1_f = np.append(r1_f,r1f)
  r2_f = np.append(r2_f,r2f)
  rl_f = np.append(rl_f,rlf)
  r1_f_ind = np.append(r1_f_ind,r1f_ind)
  r2_f_ind = np.append(r2_f_ind,r2f_ind)
  rl_f_ind = np.append(rl_f_ind,rlf_ind)

In [ ]:
print(f"\n\nAvarage Rougue-1 f1 score : { np.average(r1_f) }")
print(f"Avarage Rougue-2 f1 score : { np.average(r2_f) }")
print(f"Avarage Rougue-l f1 score : { np.average(rl_f) }")
print(f"\n\nAvarage Rougue-1 f1 score indipendence method : { np.average(r1_f_ind) }")
print(f"Avarage Rougue-2 f1 score  indipendence method : { np.average(r2_f_ind) }")
print(f"Avarage Rougue-l f1 score indipendence method : { np.average(rl_f_ind) }")

#Demo

In [7]:
from utils import Ensemble

In [27]:
text = "This paper aims to investigate the extraction of highlights from text utilizing an extractive sentence-based summarization approach. The proposed methodology involves the application of the Transformer-based Highlights Extractor (THExt) model that utilizes contextualized embeddings and transformer models to extract significant paper highlights. The THExt model is fine-tuned using the CNN Daily Email dataset to assess the generalization capability of the proposed method. Additionally, our study proposes an ensemble model that combines non-transformer-based techniques with the THExt model to enhance its effectiveness."

In [ ]:
rf = Ensemble('RandomForest')

In [28]:
rf.load("random")

In [30]:
rf.summary(text)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['The THExt model is fine-tuned using the CNN Daily Email dataset to assess the generalization capability of the proposed method.',
 'Additionally, our study proposes an ensemble model that combines non-transformer-based techniques with the THExt model to enhance its effectiveness.',
 'This paper aims to investigate the extraction of highlights from text utilizing an extractive sentence-based summarization approach.']